In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
import shap

#### Load data and split dataset

In [3]:
df = pd.read_csv('./JM006_0901_whole.csv')
df = df.dropna(subset=['milkweightlbs'])
df = df.dropna(subset=['cells'])
df = df.reset_index(drop=True)

In [4]:
disease_group = df[df['disease'] == 1]
unique_disease_cow_ids = disease_group['cow_id'].unique()
train_cow_ids, test_cow_ids = train_test_split(unique_disease_cow_ids, test_size=0.25, random_state=42)
train_disease = disease_group[disease_group['cow_id'].isin(train_cow_ids)]
#train_disease = train_disease[df['disease_in'] > 14]
#train_disease = train_disease[df['disease_in'] <= 14][df['disease_in'] >= 11]
#train_disease = train_disease[df['disease_in'] <= 11][df['disease_in'] >= 8]
#train_disease = train_disease[df['disease_in'] <= 7][df['disease_in'] >= 6]
#train_disease = train_disease[df['disease_in'] <= 5][df['disease_in'] >= 4]
#train_disease = train_disease[df['disease_in'] == 3]
#train_disease = train_disease[df['disease_in'] == 2]
train_disease = train_disease[df['disease_in'] == 1]
test_disease = disease_group[disease_group['cow_id'].isin(test_cow_ids)]
#test_disease = test_disease[df['disease_in'] > 14]
#test_disease = test_disease[df['disease_in'] <= 14][df['disease_in'] >= 11]
#test_disease = test_disease[df['disease_in'] <= 11][df['disease_in'] >= 8]
#test_disease = test_disease[df['disease_in'] <= 7][df['disease_in'] >= 6]
#test_disease = test_disease[df['disease_in'] <= 5][df['disease_in'] >= 4]
#test_disease = test_disease[df['disease_in'] == 3]
#test_disease = test_disease[df['disease_in'] == 2]
test_disease = test_disease[df['disease_in'] == 1]

health_group = df[df['disease'] == 0]
unique_health_cow_ids = health_group['cow_id'].unique()
selected_train_health_cow_ids = np.random.choice(unique_health_cow_ids, size=len(train_cow_ids), replace=False)
selected_test_health_cow_ids = np.random.choice(
    [id for id in unique_health_cow_ids if id not in selected_train_health_cow_ids],
    size=len(test_cow_ids),
    replace=False
)
train_health = health_group[health_group['cow_id'].isin(selected_train_health_cow_ids)]
test_health = health_group[health_group['cow_id'].isin(selected_test_health_cow_ids)]
avg_train_rows = train_disease.groupby('cow_id').size().mean()
avg_test_rows = test_disease.groupby('cow_id').size().mean()
train_health = train_health.groupby('cow_id').apply(lambda x: x.sample(n=int(avg_train_rows), replace=True)).reset_index(drop=True)
test_health = test_health.groupby('cow_id').apply(lambda x: x.sample(n=int(avg_test_rows), replace=True)).reset_index(drop=True)
train_health = train_health.sample(n=train_disease.shape[0], random_state=42)
test_health = test_health.sample(n=test_disease.shape[0], random_state=42)

In [5]:
X_train = pd.concat([train_disease, train_health])
X_train_np = np.hstack((X_train.iloc[:,1:488].values[:, ::-1], X_train['milkweightlbs_sca'].values.reshape(-1, 1), X_train['cells_sca'].values.reshape(-1, 1), X_train['parity_sca'].values.reshape(-1, 1)))
y_train = X_train['disease']

X_test = pd.concat([test_disease, test_health])
X_test_np = np.hstack((X_test.iloc[:,1:488].values[:, ::-1], X_test['milkweightlbs_sca'].values.reshape(-1, 1), X_test['cells_sca'].values.reshape(-1, 1), X_test['parity_sca'].values.reshape(-1, 1)))
y_test = X_test['disease']

#### Define model

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.optimizers import Adam

class MLPClassifierCustom(BaseEstimator, ClassifierMixin):
    def __init__(self, input_shape, hidden_layer_sizes=(100,), activation='relu', epochs=200, batch_size=32):
        self.input_shape = input_shape  
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = self._build_model()
        
    def _build_model(self):
        model = Sequential()
        model.add(Dense(self.hidden_layer_sizes[0], activation=self.activation, input_shape=(self.input_shape,)))
        
        for layer_size in self.hidden_layer_sizes[1:]:
            model.add(Dense(layer_size, activation=self.activation))
            
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self
    
    def predict_proba(self, X):
        return self.model.predict(X)
    
    def predict(self, X):
        y_pred = self.model.predict(X)
        return (y_pred > 0.5).astype(int).flatten()

#### Train and test model

In [7]:
clf = MLPClassifierCustom(input_shape=X_train_np.shape[1], hidden_layer_sizes=(32, 128, 32)) # 200, 200, 400, 64
clf.fit(X_train_np, y_train) 
clf.predict_proba(X_test_np) 

# training set
predicted_outcome = clf.predict(X_train_np).tolist()
true_outcome = y_train.tolist()
TP = sum([1 for p, t in zip(predicted_outcome, true_outcome) if p == 1 and t == 1])
TN = sum([1 for p, t in zip(predicted_outcome, true_outcome) if p == 0 and t == 0])
FP = sum([1 for p, t in zip(predicted_outcome, true_outcome) if p == 1 and t == 0])
FN = sum([1 for p, t in zip(predicted_outcome, true_outcome) if p == 0 and t == 1])

accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print(f"Accuracy: {accuracy}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

# test set
predicted_outcome = clf.predict(X_test_np).tolist()
true_outcome = y_test.tolist()
TP = sum([1 for p, t in zip(predicted_outcome, true_outcome) if p == 1 and t == 1])
TN = sum([1 for p, t in zip(predicted_outcome, true_outcome) if p == 0 and t == 0])
FP = sum([1 for p, t in zip(predicted_outcome, true_outcome) if p == 1 and t == 0])
FN = sum([1 for p, t in zip(predicted_outcome, true_outcome) if p == 0 and t == 1])

accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print(f"Accuracy: {accuracy}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

7/7 [==============================] - 0s 428us/step
Accuracy: 0.7546296296296297
Sensitivity: 0.6574074074074074
Specificity: 0.8518518518518519
3/3 [==============================] - 0s 621us/step
Accuracy: 0.8076923076923077
Sensitivity: 0.6410256410256411
Specificity: 0.9743589743589743


#### Get the feature importance from test set

In [11]:
clf = MLPClassifierCustom(input_shape=X_train_np.shape[1], hidden_layer_sizes=(32, 128, 32)) # 200, 200, 400, 64
clf.fit(X_train_np, y_train)  

explainer = shap.GradientExplainer(clf.model, [X_train_np, y_train])
shap_values = explainer.shap_values(X_test_np)  
shap_values = np.array(shap_values)
shap_values = shap_values.reshape(X_test_np.shape[0], X_test_np.shape[1])
shap_values

array([[0.00097806, 0.00111117, 0.00137647, ..., 0.09970991, 0.021165  ,
        0.09923735],
       [0.00060821, 0.00065856, 0.00081852, ..., 0.13882788, 0.21855034,
        0.07586302],
       [0.00057291, 0.00042864, 0.00061508, ..., 0.11296591, 0.01443004,
        0.05450489],
       ...,
       [0.00073105, 0.00053838, 0.00068968, ..., 0.06925765, 0.02641927,
        0.0799501 ],
       [0.00088795, 0.00072569, 0.00075262, ..., 0.25082266, 0.03399358,
        0.20403323],
       [0.0003819 , 0.000388  , 0.00065671, ..., 0.2128078 , 0.04314329,
        0.04142768]])

#### Get the predictive probability of test set

In [10]:
clf.predict_proba(X_test_np) 

3/3 [==============================] - 0s 1ms/step


array([[0.9329765 ],
       [0.89401335],
       [0.43075615],
       [0.7833198 ],
       [0.68400973],
       [0.6231455 ],
       [0.18733844],
       [0.7760402 ],
       [0.92181206],
       [0.9422417 ],
       [0.38584274],
       [0.9940436 ],
       [0.8154259 ],
       [0.6417288 ],
       [0.47880995],
       [0.23390047],
       [0.9043546 ],
       [0.30890784],
       [0.2985193 ],
       [0.9104303 ],
       [0.9094698 ],
       [0.7229282 ],
       [0.06205786],
       [0.71774477],
       [0.9147808 ],
       [0.20482156],
       [0.32362768],
       [0.989471  ],
       [0.70395577],
       [0.47836742],
       [0.91591823],
       [0.28025502],
       [0.792959  ],
       [0.3854156 ],
       [0.87864107],
       [0.91199774],
       [0.8506606 ],
       [0.6040987 ],
       [0.27852225],
       [0.08057581],
       [0.8549368 ],
       [0.37218174],
       [0.5187405 ],
       [0.3088027 ],
       [0.0782803 ],
       [0.2189287 ],
       [0.2065774 ],
       [0.126